### Рспознавание похожих предметов

In [2]:
import os
from glob import glob
from pymystem3 import Mystem
import sqlite3
import pandas as pd
from pyspark.sql import SQLContext, SparkSession
from pyspark import pandas as ps

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [5]:
m = Mystem()
PATH_DD = '/home/al/Projects_My/group_mlada/'
#PATH_DD = '/home/gansiorag/MyParth/ProjectMy/hakaton_ozon_org/'

In [6]:
db_path = PATH_DD + 'dataset/ozon.db'
query = """SELECT name 
FROM sqlite_schema
WHERE type ='table' AND 
    name NOT LIKE 'sqlite_%';"""

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())


[('products',)]


In [7]:
query = """SELECT name FROM PRAGMA_TABLE_INFO('products');"""
cur.execute(query)
for pp in cur.fetchall():
    print(pp[0])

search_nm
item_num
link
short_characteristic
full_desc
img_desc
img_path_desc
characteristic
components
conn_dev
img_path


In [8]:
query = """SELECT search_nm, item_num, link, short_characteristic,full_desc,
img_desc,
img_path_desc,
characteristic,
components,
conn_dev,
img_path 
FROM products;"""
a_pandas_df = pd.read_sql_query(query, conn)

In [9]:
a_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   search_nm             4398 non-null   object
 1   item_num              4398 non-null   object
 2   link                  4398 non-null   object
 3   short_characteristic  4398 non-null   object
 4   full_desc             4398 non-null   object
 5   img_desc              4398 non-null   object
 6   img_path_desc         4398 non-null   object
 7   characteristic        4398 non-null   object
 8   components            4398 non-null   object
 9   conn_dev              4398 non-null   object
 10  img_path              4398 non-null   object
dtypes: object(11)
memory usage: 378.1+ KB


In [10]:
spark = SparkSession.builder.appName('all_dataset').getOrCreate()
print('Spark info :')
spark


24/08/13 14:04:09 WARN Utils: Your hostname, al-Dell resolves to a loopback address: 127.0.1.1; using 192.168.0.199 instead (on interface enp60s0)
24/08/13 14:04:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/13 14:04:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark info :


In [9]:
a_spark_df = ps.from_pandas(a_pandas_df)
df = a_spark_df.to_spark()

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [10]:
df.show(5)

24/08/13 08:09:33 WARN TaskSetManager: Stage 0 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
|search_nm|  item_num|                link|short_characteristic|           full_desc|img_desc|       img_path_desc|      characteristic|          components|conn_dev|            img_path|
+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
| смартфон|1409765841|https://www.ozon....|О товаре Перейти ...|Описание Смартфон...|        |dataset/смартфон/...|Характеристики До...|Смартфон, кабель ...|        |dataset/смартфон/...|
| смартфон|1624251355|https://www.ozon....|О товаре Перейти ...|Комплектация Комп...|        |                    |Характеристики До...|Комплект поставки...|        |dataset/смартфон/...|
| смартфон|1337138963|https://www.ozon....|О товаре Перейти 

In [11]:
example_row_full_desc = df.select(['full_desc', 'link']).collect()


24/08/13 08:09:43 WARN TaskSetManager: Stage 1 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
for rr in range(110, 120):
    print(example_row_full_desc[rr]['full_desc'], example_row_full_desc[rr]['link'])

Описание Смартфон Xiaomi Redmi 13C 8/256GB Midnight Black работает на системе Android 13 за счет восьмиядерного процессора. Разрешение экрана составляет 720х1612 пикс. Дисплей оснащен технологией IPS. Оперативная память — 8 Гб, встроенная — 256 Гб. На задней части устройства расположены две основные камеры, позволяющие осуществлять видеосъемку с разрешением 1920х1080 пикс. Устойчивость кадра обеспечивается цифровой стабилизацией. Фронтальная камера — 8 Мп. Для увеличения емкости хранилища данных используется флэш-накопитель microSD с максимальным объемом 512 Гб. Возможно использовать при работе две nano-SIM. Передача данных осуществляется при поддержке стандартов 2G, 3G, 4G LTE, Wi-Fi, NFC, Miracast, Bluetooth 5.0. Доступны функции спутниковой навигации GPS, ГЛОНАСС, встроен FM-тюнер. На задней части панели размещен сканер отпечатка пальца. Корпус устройства сделан из пластика, размещает разъемы 3,5 мм и USB Type-C. В комплект включены блок питания мощностью 10 Вт и провод для зарядки.

In [13]:
type_obj = df.select('search_nm').distinct().collect()
for i in type_obj:
    print(i['search_nm'])

24/08/13 08:10:43 WARN TaskSetManager: Stage 2 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


смартфон
планшет
ноутбук
кроссовки
телевизор
туфли
стул
стол
платье
наушники
чехол для телефона


In [14]:
stop_words = []
with open('/home/al/Projects_My/hakaton_ozon_org/dataset/stop_word_base.txt', 'r') as f:
    for word in f.readlines():
        stop_words.append(word.strip())
len(stop_words)

5499

In [15]:
def func(value):
    for i in range(10):
        value = value.replace(str(i),'')
    value = value.replace('(-)', ' ')
    ddel = ['*', ',', '-', '/', '%', ';', ')', '(', '+', '.', '..', ':', '"',
 'c',
 'h',
 '«',
 '°',
  '–',
 '—',
 '•']
    for i in ddel:
        value = value.replace(i, ' ')
    return value

In [16]:
def get_array_text_with_category(all_text: str, m):
    """ Списка слов из списка текстов

    Args:
        all_text_list (list): Список текстов
        m (_type_): _description_
    """
    sum_arr = []
    words_list = func(all_text.lower()).split()
    for word in words_list:
        # print(word)
        word = word.strip()
        if word:
            lemmas = m.lemmatize(word)
            # print('lemmas[0]', lemmas[0])
            if lemmas[0] not in stop_words:
                print('lemmas[0]', lemmas[0])
                sum_arr.append(lemmas[0])
    return sum_arr

## Получение списков слов по объектам в каждой категориии и создаие наборов train и test.

#### План
1. Распознавание категориальное <br>
2. Распознавание внутрикатегориальное <br>
     <br>
    - по каждой категории свой набор <br>
    - внутри каждой категории наборы по  <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        с цифрами и без цифр <br> <br>
    <b>распознавание по 9 наборам слов - </b><br>
        общий <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        short_characteristic <b>с цифрами </b><br>
        full_desc <b>с цифрами </b><br>
        characteristic <b>с цифрами </b><br>
        components <b>с цифрами </b><br>
         <br>

In [17]:
def get_array_words(ddf: dict, m):
    """ Список слов в каждой категории

    Args:
        dict_txt_cat (dict): Словарь категорий
    """
    dict_cat_words = []
    # print(ddf['short_characteristic'])
    dict_cat_words += get_array_text_with_category(ddf['short_characteristic'], m)
    dict_cat_words += get_array_text_with_category(ddf['full_desc'], m)
    dict_cat_words += get_array_text_with_category(ddf['characteristic'], m)
    dict_cat_words += get_array_text_with_category(ddf['components'], m)
    dict_cat_words = set(dict_cat_words)
    return dict_cat_words


In [18]:
len(type_obj)

11

### Получение наборов слов по объектам за вычетом стоп слов

Получение общего словаря.

Получение набора с категориями

In [19]:
array_txt = [] # предложения из лемматизированных слов объектов
m = Mystem()
kk = 0
catY = [] # номера категорий
keys_cat = [] # названия категорий
all_words_list = [] # полный список слов
for sd in type_obj:
    # print(sd )
    kkd = sd['search_nm']
    keys_cat.append(kkd)
    print(kkd)
    list_dir_two = df.filter(df['search_nm'] == kkd).collect()
    # print(len(list_dir_two))
    # print(list_dir_two[10])
    for nf in list_dir_two:
        dd = get_array_words(nf, m)
        all_words_list += dd
        #break
        array_txt.append(' '.join(dd))
        catY.append(kk)
    # print()
    # print(array_txt[kk])
    kk += 1
set_all_words = set(all_words_list)
print(keys_cat)
print(len(array_txt))
print(catY)
print(len(set_all_words))

смартфон


24/08/13 08:11:49 WARN TaskSetManager: Stage 5 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] макро
lemmas[0] моноблок
lemmas[0] конвертер
lemmas[0] моноблок
lemmas[0] стабилизация
lemmas[0] ростест
lemmas[0] конвертер
lemmas[0] моноблок
lemmas[0] тср
lemmas[0] стабилизация
lemmas[0] портретный
lemmas[0] портретный
lemmas[0] реверсивный
lemmas[0] infinity
lemmas[0] каплевидный
lemmas[0] стабилизация
lemmas[0] моноблок
lemmas[0] моноблок
lemmas[0] телеобъектив
lemmas[0] стабилизация
lemmas[0] ois
lemmas[0] стабилизация
lemmas[0] sju
lemmas[0] ростест
lemmas[0] стабилизация
lemmas[0] моноблок
lemmas[0] ростест
lemmas[0] конвертер
lemmas[0] выводиться
lemmas[0] марафонский
lemmas[0] выносливость
lemmas[0] портретный
lemmas[0] кольцевой
lemmas[0] моноблок
lemmas[0] стабилизация
lemmas[0] vivo
lemmas[0] поликарбонатный
lemmas[0] моноблок
lemmas[0] vivo
lemmas[0] global
lemmas[0] моноблок
lemmas[0] телеобъектив
lemmas[0] стабилизация
lemmas[0] ois
lemmas[0] стабилизация
lemmas[0] sju
lemmas[0] ростест
lemmas[0] выводиться
lemmas[0] марафонский
lemmas[0] выносливость
lemmas[

24/08/13 08:11:53 WARN TaskSetManager: Stage 6 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] примечательный
lemmas[0] qwerty
lemmas[0] umiio
lemmas[0] umiio
lemmas[0] umiio
lemmas[0] qwerty
lemmas[0] mta
lemmas[0] zunyi
lemmas[0] анонсирование
lemmas[0] xpro
lemmas[0] mtk
lemmas[0] standby
lemmas[0] last
lemmas[0] анонсирование
lemmas[0] spreadtrum
lemmas[0] функционирование
lemmas[0] spreadtrum
lemmas[0] анонсирование
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] uwi
lemmas[0] xpro
lemmas[0] ·

lemmas[0] ·

lemmas[0] |

lemmas[0] |

lemmas[0] multitou
lemmas[0] |

lemmas[0] ·

lemmas[0] ·

lemmas[0] gmp
lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] ·

lemmas[0] uwi
lemmas[0] xpro
lemmas[0] анонсирование
lemmas[0] водозащищенный
lemmas[0] анонсирование
lemmas[0] 🎬
lemmas[0] widevine
lemmas[0] widevine
lemmas[0] тепловыделение
lemmas[0] trans
lemmas[0] анонсирование
lemmas[0] last
lemmas[0] анонсирование
lemmas[0] м

...

lemmas[0] узорчатый
lemmas[0] волнистость
lemmas[0] беззаботный
lemmas[0] олицетворять
lemmas[0] веселие
lemmas[0] щадить
lemmas[0] хлоросодержащий
lemmas[0] муслин
lemmas[0] муслин
lemmas[0] ginger
lemmas[0] декольте
lemmas[0] size
lemmas[0] обхват
lemmas[0] талия
lemmas[0] обхват
lemmas[0] грудь
lemmas[0] обхват
lemmas[0] муслин
lemmas[0] муслин
lemmas[0] ог
lemmas[0] ог
lemmas[0] покрой
lemmas[0] отложной
lemmas[0] ворот
lemmas[0] вверху
lemmas[0] талия
lemmas[0] натали
lemmas[0] ог
lemmas[0] покрой
lemmas[0] ростовка
lemmas[0] индиго
lemmas[0] отбеливание
lemmas[0] запрещать
lemmas[0] эластан
lemmas[0] эластан
lemmas[0] кардиган
lemmas[0] жакет
lemmas[0] треугольный
lemmas[0] zolla
lemmas[0] сарафанов
lemmas[0] комбинезон
lemmas[0] рубашка
lemmas[0] ог
lemmas[0] эластан
lemmas[0] эластан
lemmas[0] ог
lemmas[0] zolla
lemmas[0] покрой
lemmas[0] отбеливать
lemmas[0] барабан
lemmas[0] глажение
lemmas[0] утюг
lemmas[0] глажение
lemmas[0] необратимый
lemmas[0] тетрахлорэтилен
lemmas[0] р

24/08/13 08:12:56 WARN TaskSetManager: Stage 14 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] aylou
lemmas[0] aylou
lemmas[0] aylou
lemmas[0] сверхнизкий
lemmas[0] моно
lemmas[0] уха
lemmas[0] бежать
lemmas[0] марафон
lemmas[0] усиленно
lemmas[0] потренироваться
lemmas[0] присваиваться
lemmas[0] 💫
lemmas[0] ушной
lemmas[0] раковина
lemmas[0] автоподстройка
lemmas[0] амбушюр
lemmas[0] биопластик
lemmas[0] остановка
lemmas[0] повтор
lemmas[0] штекер
lemmas[0] импеданс
lemmas[0] aylou
lemmas[0] внутриканальный
lemmas[0] freebuds
lemmas[0] цепляться
lemmas[0] амбушюр
lemmas[0] излучатель
lemmas[0] композит
lemmas[0] транспорт
lemmas[0] объявление
lemmas[0] аэропорт
lemmas[0] вокзал
lemmas[0] слышно
lemmas[0] freebuds
lemmas[0] диссонанс
lemmas[0] дважды
lemmas[0] групповой
lemmas[0] life
lemmas[0] внутриканальный
lemmas[0] ушной
lemmas[0] раковина
lemmas[0] multipoint
lemmas[0] амбушюр
lemmas[0] остановка
lemmas[0] freebuds
lemmas[0] всенаправленный
lemmas[0] lda
lemmas[0] амброшур
lemmas[0] интсрукция
lemmas[0] внутриканальный
lemmas[0] режий
lemmas[0] индикация
lemmas[0

24/08/13 08:13:11 WARN TaskSetManager: Stage 15 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


lemmas[0] аквакейс
lemmas[0] герметичность
lemmas[0] опаска
lemmas[0] аквакейс
lemmas[0] гермочехол
lemmas[0] кредитный
lemmas[0] аквакейс
lemmas[0] реалми
lemmas[0] реалми
lemmas[0] реалми
lemmas[0] картхолдер
lemmas[0] карбоновый
lemmas[0] поликарбонат
lemmas[0] бортик
lemmas[0] соприкасаться
lemmas[0] пожелтение
lemmas[0] поликарбонат
lemmas[0] картхолдер
lemmas[0] автомобильный
lemmas[0] asai
lemmas[0] поликарбонат
lemmas[0] банковский
lemmas[0] скидочный
lemmas[0] проездной
lemmas[0] бортик
lemmas[0] kjs
lemmas[0] inf
lemmas[0] trn
lemmas[0] wallet
lemmas[0] over
lemmas[0] ef
lemmas[0] wa
lemmas[0] begru
lemmas[0] бортик
lemmas[0] возвышаться
lemmas[0] olourful
lemmas[0] zes
lemmas[0] хлястик
lemmas[0] редми
lemmas[0] банковский
lemmas[0] визитка
lemmas[0] новаякнижка
lemmas[0] картхолдер
lemmas[0] влияние
lemmas[0] бортик
lemmas[0] обгонять
lemmas[0] дубеть
lemmas[0] плавиться
lemmas[0] противоразводный
lemmas[0] прилипать
lemmas[0] банковский
lemmas[0] банковский
lemmas[0] диско

In [20]:
array_txt

['макро моноблок',
 'ростест стабилизация конвертер моноблок',
 'тср моноблок',
 'реверсивный стабилизация портретный каплевидный infinity моноблок',
 'стабилизация ois sju ростест моноблок телеобъектив',
 'ростест стабилизация конвертер моноблок',
 'портретный стабилизация выносливость марафонский кольцевой выводиться моноблок',
 'global vivo моноблок поликарбонатный',
 'стабилизация ois sju ростест моноблок телеобъектив',
 'портретный стабилизация выносливость марафонский кольцевой выводиться моноблок',
 'ростест стабилизация моноблок',
 'моноблок',
 'виджет ростест pdaf atgpt исход поэзия моноблок ”\n светочувствительный разгон трехкольцевой darwin аудиовселенная стабилизация знакомиться slimming listen \ufeff\ufeff ー\n',
 'буклет лот стабилизация моноблок',
 'вибромотор осевой стабилизация сверхъемкий dimensity жидкостный лампочка meteorite [\n ростест моноблок ー\n',
 'ростест стабилизация моноблок',
 'стабилизация неавторизованный вашы ростест моноблок',
 'голосование победитель т

In [1]:
catY[0]

NameError: name 'catY' is not defined

In [23]:
keys_cat[0]

'смартфон'

In [24]:
X_train, X_test, y_train, y_test = train_test_split(array_txt, catY, test_size=0.20, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [25]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [26]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [27]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [28]:
kk = 0
for doc, category in zip(y_test, predicted):
    # print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

model  0.8920454545454546


In [29]:
vectorizer.get_feature_names_out()

array(['aaa', 'aaax', 'aax', ..., 'ячменный', 'ящ', 'ящичек'],
      dtype=object)

In [30]:
print(X_test_new.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Построение распознавания внутри категорий

#### без цифр

In [31]:
def get_array_text_with_category2(all_text: str, m):
    """ Списка слов из списка текстов

    Args:
        all_text_list (list): Список текстов
        m (_type_): _description_
    """
    sum_arr = []
    words_list = func(all_text.lower()).split()
    for word in words_list:
        # print(word)
        word = word.strip()
        if word:
            lemmas = m.lemmatize(word)
            # print('lemmas[0]', lemmas[0])
            # if lemmas[0] not in stop_words:
            #     print('lemmas[0]', lemmas[0])
            sum_arr.append(lemmas[0])
    return sum_arr

In [32]:
def get_array_text_with_category_param(df, m, cat: str, param: str):
    list_obj_cat = df.filter(df['search_nm'] == cat).collect()
    array_short_characteristic = []
    num_oby = []
    k = 0
    for obj in list_obj_cat:
        print(obj[param])
        list_words = get_array_text_with_category2(obj[param], m)
        print(list_words)
        text_words = ' '.join(list_words)
        array_short_characteristic.append(text_words)
        num_oby.append(k)
        k += 1
    return array_short_characteristic, num_oby, list_obj_cat

In [33]:
cat_desc = ['short_characteristic', 'full_desc', 'characteristic', 'components']

In [34]:
array_short_characteristic, num_oby, list_obj_cat = get_array_text_with_category_param(df, m, 'смартфон', 'short_characteristic')
full_desc, num_oby1, list_obj_cat1 = get_array_text_with_category_param(df, m, 'смартфон', 'full_desc')
characteristic, num_oby2, list_obj_cat2 = get_array_text_with_category_param(df, m, 'смартфон', 'characteristic')
components, num_oby3, list_obj_cat3 = get_array_text_with_category_param(df, m, 'смартфон', 'components')

24/08/13 08:23:14 WARN TaskSetManager: Stage 16 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


О товаре Перейти к описанию Тип Смартфон Диагональ экрана, дюймы 6.67 Емкость аккумулятора, мАч 5000 Процессор Snapdragon 685 (8 ядер), 2.8 ГГц Основной материал корпуса Пластик, Стекло
['о', 'товар', 'переходить', 'к', 'описание', 'тип', 'смартфон', 'диагональ', 'экран', 'дюйм', 'емкость', 'аккумулятор', 'мач', 'процессор', 'snapdragon', 'ядро', 'ггц', 'основной', 'материал', 'корпус', 'пластик', 'стекло']
О товаре Перейти к характеристикам Тип Смартфон Диагональ экрана, дюймы 7.3 Емкость аккумулятора, мАч 5500 Процессор Snapdragon 8 Gen1 (8 ядер), 3 ГГц Основной материал корпуса Металл, Стекло
['о', 'товар', 'переходить', 'к', 'характеристика', 'тип', 'смартфон', 'диагональ', 'экран', 'дюйм', 'емкость', 'аккумулятор', 'мач', 'процессор', 'snapdragon', 'gen', 'ядро', 'ггц', 'основной', 'материал', 'корпус', 'металл', 'стекло']
О товаре Перейти к описанию Тип Смартфон Диагональ экрана, дюймы 6.74 Емкость аккумулятора, мАч 5000 Процессор Helio G85 (8 ядер), 2.0 ГГц Основной материал кор

24/08/13 08:23:14 WARN TaskSetManager: Stage 17 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


Описание Смартфон Xiaomi Redmi Note 13 Redmi Note 13 - это мощный и современный телефон, который предлагает широкий набор функций и характеристик для удовлетворения потребностей пользователей. Он оснащен процессором Snapdragon® 6856 процессором, что обеспечивает высокую производительность и плавную работу. В Redmi Note 13 имеется несколько вариантов хранилища и оперативной памяти: 6 ГБ + 128 ГБ, 8 ГБ + 128 ГБ и 8 ГБ + 256 ГБ. Кроме того, имеется возможность расширения хранилища до 1 ТБ, что позволяет сохранить большое количество данных, фотографий и видео. Дисплей телефона имеет размер 6,67 дюйма и использует технологию AMOLED. Он поддерживает частоту обновления до 120 Гц, что обеспечивает плавное отображение контента. Камера заднего вида Redmi Note 13 предлагает тройную систему: 108 МП основная камера, 8 МП сверхширокоугольная камера и 2 МП макро-камера. Аккумулятор Redmi Note 13 имеет емкость 5000 мАч и поддерживает быструю зарядку мощностью 33 Вт. Телефон также обладает функцией без

24/08/13 08:23:16 WARN TaskSetManager: Stage 18 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


Характеристики Добавить к сравнению Экран Диагональ экрана, дюймы 6.67 Разрешение экрана 2400x1080 Технология матрицы AMOLED Заводские данные Степень защиты IP54 Память и процессор Процессор Snapdragon 685 (8 ядер), 2.8 ГГц Число ядер процессора 8 Оперативная память 6 ГБ Встроенная память 128 ГБ Тип карты памяти microSD Бренд процессора Qualcomm Модель процессора Snapdragon 685 Частота процессора, ГГц 2.8 Бренд графического процессора Qualcomm Видеопроцессор Adreno 610 Общие Емкость аккумулятора, мАч 5000 Основной материал корпуса Пластик , Стекло Операционная система Android Версия Android Android 13 Тип корпуса Моноблок Размеры, мм 162,2 x 75,5 x 8 Вес товара, г 188 Гарантийный срок 1 год Функции Аутентификация Распознавание лица , Сканер отпечатка Интерфейсы 3.5 мм , USB Type-C Слот для карты памяти Да Функции зарядки Поддержка быстрой зарядки Встроенные датчики : Сканер отпечатка пальца , Датчик распознавания лица , Датчик приближения , Датчик освещенности Связь Беспроводные интерф

24/08/13 08:23:17 WARN TaskSetManager: Stage 19 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


Смартфон, кабель для зарядки, документация
['смартфон', 'кабель', 'для', 'зарядка', 'документация']
Комплект поставки: мобильный телефон + упаковочная коробка + руководство пользователя + зарядное устройство + кабель для передачи данных Type-C + гарнитура Type-C + защитная пленка для мобильного телефона + защитный чехол + pin-код карты + OTG-конвертер.
['комплект', 'поставка', 'мобильный', 'телефон', 'упаковочная', 'коробка', 'руководство', 'пользователь', 'зарядный', 'устройство', 'кабель', 'для', 'передача', 'данный', 'type', 'гарнитур', 'type', 'защитный', 'пленка', 'для', 'мобильный', 'телефон', 'защитный', 'чехол', 'pin', 'код', 'карта', 'otg', 'конвертер']
Смартфон, адаптер питания, кабель USB Type-C, инструмент для извлечения SIM-карты, руководство пользователя, гарантийный талон
['смартфон', 'адаптер', 'питание', 'кабель', 'usb', 'type', 'инструмент', 'для', 'извлечение', 'sim', 'карта', 'руководство', 'пользователь', 'гарантийный', 'талон']
Документация, зарядное устройство, к

In [35]:
array_short_characteristic

['о товар переходить к описание тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор snapdragon ядро ггц основной материал корпус пластик стекло',
 'о товар переходить к характеристика тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор snapdragon gen ядро ггц основной материал корпус металл стекло',
 'о товар переходить к описание тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор elio g ядро ггц основной материал корпус пластик',
 'о товар переходить к описание тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор elio g ядро ггц основной материал корпус пластик',
 'о товар переходить к описание тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор snapdragon gen ядро ггц основной материал корпус стекло металл',
 'о товар переходить к описание тип смартфон диагональ экран дюйм емкость аккумулятор мач процессор snapdragon gen ядро ггц основной материал корпус металл стекло',
 'о товар переходить к описание тип

In [36]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from time import time

from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [46]:
def get_clasters(nabor, num_oby):
    X_train_sh, X_test_sh, y_train_sh, y_test_sh = train_test_split(nabor, num_oby, test_size=0.20, random_state=42)
    vectorizer_sh = CountVectorizer()
    X_train_counts_sh = vectorizer_sh.fit_transform(X_train_sh)
    # X_train_counts_sh    
    kmeans = KMeans(
        n_clusters= 20, # int(len(y_train_sh)),
        max_iter=5,
        # n_init=1,
        # random_state=2,
    ).fit(X_train_counts_sh)
    ones = {}
    for i, y in zip(kmeans.labels_, y_train_sh):
        if i not in ones:
            ones[i] = []
        ones[i].append(y)
    return ones, kmeans, X_test_sh, y_test_sh, vectorizer_sh

In [50]:
def get_classes(kmeans, X_test_sh, y_train_sh, vectorizer_sh):
    # vectorizer_sh = CountVectorizer()
    X_test_sh_counts_sh = vectorizer_sh.transform(X_test_sh)
    # X_test_sh_counts_sh    
    kmeans_p = kmeans.predict(X_test_sh_counts_sh)
    ones = {}
    for i, y in zip(kmeans_p, y_train_sh):
        if i not in ones:
            ones[i] = []
        ones[i].append(y)
    return ones

In [54]:
# 'full_desc', 'characteristic', 'components'
ones, kmeans0, X_test_sh0, y_test_sh0, vectorizer_sh0 = get_clasters(array_short_characteristic, num_oby)
print(len(ones))
ones1, kmeans1, X_test_sh1, y_test_sh1, vectorizer_sh1 = get_clasters(full_desc, num_oby1)
print(len(ones1))
ones2, kmeans2, X_test_sh2, y_test_sh2, vectorizer_sh2 = get_clasters(characteristic, num_oby2)
print(len(ones2))
ones3, kmeans3, X_test_sh3, y_test_sh3, vectorizer_sh3 = get_clasters(components, num_oby3)
print(len(ones3))

/home/al/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


20
20
20
20


In [55]:
ones_short_characteristic  = get_classes(kmeans0, X_test_sh0, y_test_sh0, vectorizer_sh0)
print(ones)

{11: [16, 9, 6, 48, 88], 9: [18, 94, 45, 136, 126, 64, 39, 105, 76, 83, 73, 25, 62, 85, 34, 84, 13, 3, 17, 95, 50, 115, 90, 57, 107, 21, 2, 116, 87, 20, 106], 12: [10, 0, 133, 128, 134], 1: [111, 93, 125, 37], 13: [101], 4: [51, 112, 15, 89, 47, 30, 68, 135, 97, 54, 91, 63, 102], 2: [82, 33, 77, 5, 35, 8, 58, 124, 32, 1, 74], 17: [65, 55, 137], 8: [28, 40, 53, 49, 43, 38, 61, 122, 130, 71, 92], 3: [24, 98, 52, 103], 5: [44, 22, 29], 10: [100, 129], 7: [86, 75], 18: [80, 72, 59, 23], 0: [7, 117, 123, 46, 79, 14], 14: [70], 15: [114], 19: [41], 16: [120, 121, 139], 6: [99]}


In [57]:
ones_full_desc = get_classes(kmeans1, X_test_sh1, y_test_sh1, vectorizer_sh1)
print(len(ones_full_desc))


8


In [58]:
ones_characteristic = get_classes(kmeans2, X_test_sh2, y_test_sh2, vectorizer_sh2)
print(len(ones_characteristic))


16


In [59]:
ones_components = get_classes(kmeans3, X_test_sh3, y_test_sh3, vectorizer_sh3)
print(len(ones_components))

13


In [63]:
def groupping(ones, ones1, ones2, ones3):
    group = {}
    for k in ones:
        #print(k)
        ls = []
        for dd in ones[k]:
            #list_obj_cat[k]['link']
            if list_obj_cat[dd]['link'] not in group:
                group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones1:
        #print(k)
        ls = []
        for dd in ones1[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones1[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones2:
        #print(k)
        ls = []
        for dd in ones2[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones2[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    for k in ones3:
        #print(k)
        ls = []
        for dd in ones3[k]:
            #list_obj_cat[k]['link']
            # if list_obj_cat[dd]['link'] not in group:
            #     group[list_obj_cat[dd]['link']] = []
            ls.append(list_obj_cat[dd]['link'])
        # print(len(group))
        for d2 in ones3[k]:
            ff = [dd for dd in ls if dd !=list_obj_cat[d2]['link']]
            # print(len(ff))
            # print(ff)
            # print(list_obj_cat[d2]['link'])
            group[list_obj_cat[d2]['link']].append(set(ff))
            # print(' --> ', len(group[list_obj_cat[d2]['link']]))
    return group

In [64]:
group = groupping(ones, ones1, ones2, ones3)

In [65]:
for k in group:
    print(k)
    print(len(group[k]))
    # for dd in group[k]: 
    #     print(len(dd))

https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-belyy-1429044516/?asb=0L3ES%252FmKe9CZj3IatQE8VxtTEE0dzMEKNmLHBCaZ9PE%253D&asb2=09HHEs905z6MLvBe-TneIl_GwshpmetduAUVuyVfvzPeXtq7QhcfdpvbZdLB7N7xDqVvNAPhCodApCSGCGgrxg&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-zelenyy-1429044044/?asb=vydTW5EVIbMv4kR1RaEX%252FsX6wwC0U2FdErkE3KTyc2E%253D&asb2=KESvEYkqlxtYt4a96752IXVuV_fzn5MOLifgEi9eXdQ5Z1qyVoR7D0RRbYmMM1C335Ccs1fdMkdcqaTM0rFbTg&avtc=1&avte=2&avts=1721078417&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
4
https://www.ozon.ru/produc

In [67]:
len(group)

112

In [66]:
dde = 0
for k in group:
    #print(k)
    # # print(group[k])
    com = set.intersection(*group[k])
    # print(com)
    # com = group[k][0].intersection(group[k][2])
    # print(com)
    # com = group[k][0].intersection(group[k][3])
    # print(com)
    if len(com) > 0:
        print(k)
        for dd in com:
            print(dd)
        print()
        print('='*20)
        dde += 1
    if dde > 5: 
        break

https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-belyy-1429044516/?asb=0L3ES%252FmKe9CZj3IatQE8VxtTEE0dzMEKNmLHBCaZ9PE%253D&asb2=09HHEs905z6MLvBe-TneIl_GwshpmetduAUVuyVfvzPeXtq7QhcfdpvbZdLB7N7xDqVvNAPhCodApCSGCGgrxg&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-zelenyy-1429044044/?asb=vydTW5EVIbMv4kR1RaEX%252FsX6wwC0U2FdErkE3KTyc2E%253D&asb2=KESvEYkqlxtYt4a96752IXVuV_fzn5MOLifgEi9eXdQ5Z1qyVoR7D0RRbYmMM1C335Ccs1fdMkdcqaTM0rFbTg&avtc=1&avte=2&avts=1721078417&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-chernyy-1429044993/?asb=31YIIMqSzgw0NlJHUY17AItO1arAaT5SWAV%252Bcv%252BZYrY%253D&asb2=sFRH0h2cvxdHODkEIAGR2EwgzrHzoDhjhtmwq4zSUdcncClLsfVefWvsblxvH90-LzxTnryxNE966FHMSHcTpQ&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

https://www.ozon.ru/product/infinix-smar

In [198]:
dde = 0
for k in group:
    #print(k)
    # # print(group[k])
    com = set.intersection(*group[k])
    # print(com)
    # com = group[k][0].intersection(group[k][2])
    # print(com)
    # com = group[k][0].intersection(group[k][3])
    # print(com)
    if len(com) > 0:
        if dde > 5: 
            print(k)
            for dd in com:
                print(dd)
            print()
            print('='*20)
        dde += 1
    if dde > 10: 
        break

https://www.ozon.ru/product/xiaomi-smartfon-redmi-13c-8-256-gb-siniy-1337138965/?asb=aoVgBk4meJGoXecJMjjid7ViiLRte2AiIFVryZa828E%253D&asb2=p4yr7zXRpLr3ZyyBQJREmZCgRM2BpwcoBjCBd8JOLFmvnlYDPKZkTXTqvG-uhFDaEDywr01ksSXw-0Ghel9BSw&avtc=1&avte=1&avts=1721078427&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/xiaomi-smartfon-redmi-13c-4-128-gb-belyy-1337138963/?asb=8qY2ayBN%252B3g37op5ER6EagYgwhNnfM6U0glizuoTneQ%253D&asb2=qadtD_kXJ6ph21aS7EEd4g7CyimGkYJafWIaj-xVg1wOmLT1w51zx6hghdijNk1_KqtNdmAAq8NtaJvrKaS2GA&avtc=1&avte=1&avts=1721078453&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD

https://www.ozon.ru/product/xiaomi-smartfon-redmi-12-8-256-gb-chernyy-1125391207/?asb=n96GwvgzpGapuAyCOaes9hZGm2zWooHaJuWtwG177mo%253D&asb2=kW_q4SLSNIOBYhtb2-bhbAeE0WmGRFfLMl-ZO3qYDLtYm3CDIF7Fj3z__G6_K-40VFi7czYIT0gLRCO4CFWWBQ&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/xiaomi-smartfon-redmi-12

In [71]:
group2 = groupping(ones_short_characteristic, ones_full_desc, ones_characteristic, ones_components)
print(len(group2))

28


In [73]:
group2

{'https://www.ozon.ru/product/zunyi-smartfon-note-30i-rostest-eac-12-256-gb-salatovyy-1599077247/?advert=Al7gxXnUbdwUpvE0tbGjsa0mt_X6N5_0yPSrI1W4kHmrBL6oVacNFbpMUqBNcpwJKUvtjmwLn4646TvpF7RKmxcRePYwGCSwDn3nJ2rRqsDneeDwkV7Jrf0dfDJ6z1Vep_gCk2yJqlzpO2F9jJ0V2wpjD2XzC5GoQbXzTLZuZD_5hFba-Hj5T5nTlk03ICsltaii-5KDw4TTFPUBxRh-hJDy5eSpLkyjgPrL4VPf_m9yFyjm54CCWaFf9noTPvXZpgeb1pyY1Q3ZoyzNtA8RR7-rXS-lGeue1W8nghLzG7Heu_L2BCqhui7XzPPEnsUgPGKabPmOQiB2U-JzH3PpmUAYVxGdUxQPDw8nYfL9yKl4WJtQKurlw-cuguG3xlCjncbfD-ouLw&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD': [{'https://www.ozon.ru/product/samsung-smartfon-galaxy-s24-ultra-12-256-gb-zheltyy-1398851772/?advert=xF2wHsr4owAVkwsBh0iFhVel473tQedBfO1zVTgnzm7worQk4cD3GHP03PTkYogXFe0XIwzOUz4bln-cws4-MugEFr425WzVkXCRadibh3OadWjvURROk9q6X5oZtQ7dS_FkgaVeAYGem2nTVXEs33jW7Bb0O_TLjXwsXcf3VLbAKb9RPCnYST22G9XrPGCaWANq7Lerdoz7toYuZcl2HdzmoBGccDDed4UOSi-W59SR74xzg8zg1ecD54i2rFPjOzpXASreCMKH40-CGDnEkxeudLwmesmz3SeCTIOpn0vVQZdKKd4k

In [75]:
dde2 = 0
for k in group2:
    print(k)
    # # print(group[k])
    com = set.intersection(*group2[k])
    # print(com)
    # com = group[k][0].intersection(group[k][2])
    # print(com)
    # com = group[k][0].intersection(group[k][3])
    # print(com)
    if len(com) > 0:
        if dde2 > 0: 
            print(k)
            for dd in com:
                print(dd)
            print()
            print('='*20)
        dde2 += 1
    if dde2 > 5: 
        break

https://www.ozon.ru/product/zunyi-smartfon-note-30i-rostest-eac-12-256-gb-salatovyy-1599077247/?advert=Al7gxXnUbdwUpvE0tbGjsa0mt_X6N5_0yPSrI1W4kHmrBL6oVacNFbpMUqBNcpwJKUvtjmwLn4646TvpF7RKmxcRePYwGCSwDn3nJ2rRqsDneeDwkV7Jrf0dfDJ6z1Vep_gCk2yJqlzpO2F9jJ0V2wpjD2XzC5GoQbXzTLZuZD_5hFba-Hj5T5nTlk03ICsltaii-5KDw4TTFPUBxRh-hJDy5eSpLkyjgPrL4VPf_m9yFyjm54CCWaFf9noTPvXZpgeb1pyY1Q3ZoyzNtA8RR7-rXS-lGeue1W8nghLzG7Heu_L2BCqhui7XzPPEnsUgPGKabPmOQiB2U-JzH3PpmUAYVxGdUxQPDw8nYfL9yKl4WJtQKurlw-cuguG3xlCjncbfD-ouLw&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/zunyi-smartfon-pove-5-pro-russkaya-globalnaya-versiya-protivoudarnaya-i-vodonepronitsaemaya-1620203481/?advert=d-0ll_aJkfQ2Phjo3afAlHskna-QKufE_54ha396Vn18bo3hSDfB6Hx3cXvfS5vshZQmiVrGgkI6N5z9x109-dvXoQOfsgvVPZYQXsc_zLucZ98CQyAam4h_SOeM52QTInIQBKCwibnjmJJBpPvc9OtlcQUhBg67pRGpR5GxoTAWDNvwkXBhl26Xju0yK9SQ-eWx83gQKwl1LVpPEBC5QNN0pVbOlAxcdKhJYWEdqpmD_NH2vHgvvSUjkQTBd0BMN5cCnjpVhP0ymujENYb

In [77]:
dde3 = 0
for k in group2:
    print(k)
    # # print(group[k])
    com = set.intersection(*group2[k])
    # print(com)
    # com = group[k][0].intersection(group[k][2])
    # print(com)
    # com = group[k][0].intersection(group[k][3])
    # print(com)
    if len(com) > 0:
        if dde2 > 5: 
            print(k)
            for dd in com:
                print(dd)
            print()
            print('='*20)
        dde2 += 1
    if dde2 > 10: 
        break

https://www.ozon.ru/product/zunyi-smartfon-note-30i-rostest-eac-12-256-gb-salatovyy-1599077247/?advert=Al7gxXnUbdwUpvE0tbGjsa0mt_X6N5_0yPSrI1W4kHmrBL6oVacNFbpMUqBNcpwJKUvtjmwLn4646TvpF7RKmxcRePYwGCSwDn3nJ2rRqsDneeDwkV7Jrf0dfDJ6z1Vep_gCk2yJqlzpO2F9jJ0V2wpjD2XzC5GoQbXzTLZuZD_5hFba-Hj5T5nTlk03ICsltaii-5KDw4TTFPUBxRh-hJDy5eSpLkyjgPrL4VPf_m9yFyjm54CCWaFf9noTPvXZpgeb1pyY1Q3ZoyzNtA8RR7-rXS-lGeue1W8nghLzG7Heu_L2BCqhui7XzPPEnsUgPGKabPmOQiB2U-JzH3PpmUAYVxGdUxQPDw8nYfL9yKl4WJtQKurlw-cuguG3xlCjncbfD-ouLw&avtc=1&avte=2&avts=1721078458&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/zunyi-smartfon-pove-5-pro-russkaya-globalnaya-versiya-protivoudarnaya-i-vodonepronitsaemaya-1620203481/?advert=d-0ll_aJkfQ2Phjo3afAlHskna-QKufE_54ha396Vn18bo3hSDfB6Hx3cXvfS5vshZQmiVrGgkI6N5z9x109-dvXoQOfsgvVPZYQXsc_zLucZ98CQyAam4h_SOeM52QTInIQBKCwibnjmJJBpPvc9OtlcQUhBg67pRGpR5GxoTAWDNvwkXBhl26Xju0yK9SQ-eWx83gQKwl1LVpPEBC5QNN0pVbOlAxcdKhJYWEdqpmD_NH2vHgvvSUjkQTBd0BMN5cCnjpVhP0ymujENYb

In [76]:
for k in ones:
    print(k)
    for dd in ones[k]:
        print(list_obj_cat[dd]['link'])
    print()

11
https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-belyy-1429044516/?asb=0L3ES%252FmKe9CZj3IatQE8VxtTEE0dzMEKNmLHBCaZ9PE%253D&asb2=09HHEs905z6MLvBe-TneIl_GwshpmetduAUVuyVfvzPeXtq7QhcfdpvbZdLB7N7xDqVvNAPhCodApCSGCGgrxg&avtc=1&avte=2&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/infinix-smartfon-smart-8-4-128-gb-zelenyy-1429044044/?asb=vydTW5EVIbMv4kR1RaEX%252FsX6wwC0U2FdErkE3KTyc2E%253D&asb2=KESvEYkqlxtYt4a96752IXVuV_fzn5MOLifgEi9eXdQ5Z1qyVoR7D0RRbYmMM1C335Ccs1fdMkdcqaTM0rFbTg&avtc=1&avte=2&avts=1721078417&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/i

In [115]:
for k in ones1:
    print(k)
    for dd in ones1[k]:
        print(list_obj_cat[dd]['link'])
    print()

19
https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/xiaomi-smartfon-redmi-a3-rostest-eac-4-128-gb-zelenyy-1427319695/?asb=qywlO1Y94XLWPdcFCLl4Zv0IRLSgvKHbB8elzMajrRg%253D&asb2=x1zGQHO68IvkipYa7wONAK7ZQwBxSLjii7R16GGZWPpBNRe9bZWmGHnVF3ov26qcu1eNRL4M_p1PBIYNyl59WA&avtc=2&avte=1&avts=1721078417&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/samsung-smartfon-galaxy-a05-rostest-eac-4-64-gb-chernyy-1482732205/?asb=aDdj8AXqNImoabyO4IjhQmYqTMr77FWx%252FQflVjRdIWE%253D&asb2=-gCghT7KRxy52J0FGeXdIH-KDrhB6fMg3Tb8AfZnlHY16pr9kY_3OgD5JvH5ot4Avv3yBkHDNHSH7G0YqzUVmw&avtc=1&avte=2&avts=1721078478&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https

In [116]:
for k in ones2:
    print(k)
    for dd in ones2[k]:
        print(list_obj_cat[dd]['link'])
    print()

13
https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/realme-smartfon-c51-6-256-gb-zelenyy-1440343389/?asb=aH5KhNQWtlEO62a6meBRwDT2JSvtKPgxtv8X9mMiNG0%253D&asb2=btgsSkn7AcLse4A953RVlRrK5qzg_xKIJcLITT23eSGaI6rSZ94--2fjYWDnAeUfqT-eB4TxdkFahy9DGyZFtQ&avtc=1&avte=1&avts=1721078393&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/realme-smartfon-c51-4-64-gb-zelenyy-1290426624/?asb=R1vnHtQZ0sgv9s%252FuJtgsTVtTQ%252BqD4Hef9xeXk4dIFg8%253D&asb2=5xH-ibtY5CZtSEo2a8B0bpjx3d35Obw18HKMDWJDJDfGENNAwWb0sdqw5GkPXu2G-j5zPLYinLdVh1PEJB4W7Q&avtc=2&avte=1&avts=1721078483&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/realme-sma

In [117]:
for k in ones3:
    print(k)
    for dd in ones3[k]:
        print(list_obj_cat[dd]['link'])
    print()

13
https://www.ozon.ru/product/zentra-smartfon-sp4-rostest-eac-12-64-gb-temno-zelenyy-1601468542/?asb=EWrDgTKLYT70eYhqPMVCBHm0ED6BAUMhZfwna7I%252Bf1g%253D&asb2=0-W_lZAM1tx0as_WoZb0f2hfEOfEbS-1h9fYHeYlNKLpVCTvsBlnx3GQ9MFzFtDdacZhIgcU1d-7GvqvuR9Txw&avtc=1&avte=4&avts=1721078463&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/xiaomi-smartfon-redmi-13c-8-256gb-black-8-256-gb-chernyy-1404624083/?asb=q3z4ldGiHaz2mQMaSrpCyRQZ%252F8UYvjoViNiTWOSTQH8%253D&asb2=npaYzwqqyqBwtJkVqJ4gy_oKuvWCz17qU1E11vpclIucSv1jWVKTDg4CF5M2To0Fy5pg4dLWLXWB7pwEE3yepQ&avtc=1&avte=2&avts=1721078448&keywords=%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD
https://www.ozon.ru/product/honor-smartfon-90-lite-8-256-gb-goluboy-1371891666/?advert=G97ibJri9jvbS2_jrTGcqrzWRLWBPK-Fh8ldjGlKdhKDQ1M4s44idS7mDMp4gm1HTxkHXuU8HEKBmk_mkqeTGs7_Z-OsdL5gFr5Sja9Si_HFyOeXCJ0_00sUxlKj28qKtPl7vA-LK8KWtiYSXf7J1fk_6N2KjIfsITmx-TIEDVv9aOvfxVxypjIgpc0SSSgHt1z5ZaJEcVw4z0AkTiTZ2yXB18vXX8HaQfTvlhrd8JCUhuJPDl_T